In [35]:
import os
import sys
sys.path.insert(0, '..')

from google.cloud import bigquery
from google.oauth2 import service_account

import pandas as pd
import matplotlib.pyplot as plt
from decouple import config
import json
import numpy as np
import seaborn as sns

from sklearn.linear_model import LinearRegression

from authenticate_service_account import main
from utils import *

from sklearn.model_selection import train_test_split

from scipy import stats

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import r2_score

import category_encoders as ce
from scipy import stats

from sklearn.preprocessing import MinMaxScaler

from sklearn.neighbors import KNeighborsRegressor

from keras.models import load_model

from keras.callbacks import EarlyStopping
from tensorflow import keras
from tensorflow.keras import layers, Sequential

from joblib import dump, load

from sklearn.ensemble import RandomForestRegressor

from scipy.stats import zscore


In [81]:
def create_df_of_all_categories(raw_data, data_to_be_processed, column):
    """
    This creates a list of all categories, and appends it to existing data that needs processing
    it appends the data, processes it, and drops the unnecesary columns
    It then appends it back to the dataframe so it can be used in the next step
    """

    # Creating dummy data
    categories  = pd.DataFrame(raw_data[column].unique(), columns=[column])

    #Isolating the important data
    # data_for_processing = data_to_be_processed[[column]].copy()

    # Merged data for encoding
    merged_data = pd.concat([data_to_be_processed,categories])

    return merged_data


In [80]:
def convert_bool_to_num(value):
    """
    This converts bulian values to 0 or 1, but flips True and False
    So False will == 1, and True == 0
    This is is used when False values are seen as 'superior' to True values
    """
    return 0 if value else 1

In [2]:
client = main()

query = """
SELECT *
FROM skyscanner-insights-343713.Itinerary_Scoring.training_set_80pct_ODs
"""

query_job = client.query(query)

results = query_job.result()

In [3]:
gcp_data = results.to_dataframe()

In [4]:
gcp_data.to_csv('../raw_data/skyscanner_data_171223', index=False)

In [5]:
raw_data = gcp_data.copy()

In [ ]:
def feature_engineering(df, raw_od_columns, raw_od_ctry, as_ratio=False):
    """
    This runs all the preprocessing functions
    """

    # This creates a column to identify OD's
    df_with_od = create_od_column(df, raw_od_columns)

    df_with_od = create_od_ctry(df, raw_od_ctry)

    # This calculates the total layover time
    df_with_layover_time = calculate_total_layover_time(df_with_od, as_ratio)

    # This calculates the total distance traveled
    df_with_distance = calculate_total_distance(df_with_layover_time)

    # This calculates the difference between total distance traveled and 'straight line' distance
    df_with_distance_diff = calculate_distance_difference(df_with_distance, as_ratio)

    # This drops all rows with neg layover time
    df_final = drop_neg_layover_time(df_with_distance_diff)

    df_scaled = scale_itin_redirects(df_final, 'ItineraryRedirects', 0, 50)

    return df_scaled

In [8]:
raw_data.dtypes

OriginApt                      object
OriginCty                      object
OriginCtry                     object
DestinationApt                 object
DestinationCty                 object
DestinationCtry                object
TravelHorizonDays               Int64
TravelDistanceKm                Int64
SelfTransfer                  boolean
Stops                           Int64
DurationMin                     Int64
dayofweek                       Int64
Seg_0_OperatingCarrierIATA     object
Seg_1_OperatingCarrierIATA     object
Seg_2_OperatingCarrierIATA     object
Seg_3_OperatingCarrierIATA     object
Total_Flight_Distance           Int64
Total_Flight_Duration           Int64
passengers                      Int64
PricePerPax                   float64
ItineraryRedirects              Int64
ODRedirects                     Int64
dtype: object

In [85]:
def all_preprocessing(raw_data, columns_to_process, target_creation_function, target,
                        box_cox_columns=False, yeo_johnson_columns=False, min_max_scaling=False, log_transform_columns=False,
                        target_func_param1=None, target_func_param2=None, target_func_param3=None):
    """
    This functions completes all feature engineering, target creation and scaling
    RETURNS: updated dataframe and a Class that holds all the scalers

    Notes:
    - It will only return columns in columns_to_process and the target
    """

    #FEATURE ENGINEERING SECTION

    # This creates a column to identify OD's
    raw_data['OD'] = raw_data['OriginCty'] + raw_data['DestinationCty']

    # This calculates the total layover time with ratio
    raw_data['total_layover_time'] = raw_data['DurationMin'] - raw_data['Total_Flight_Duration']
    raw_data['total_layover_time_ratio'] =raw_data['total_layover_time'] /raw_data['DurationMin']

    # This calculates the difference between total distance traveled and 'straight line' distance
    raw_data['extra_travel_distance'] = raw_data['Total_Flight_Distance'] - raw_data['TravelDistanceKm']
    raw_data['extra_travel_distance_ratio'] =  raw_data['Total_Flight_Distance'] / raw_data['TravelDistanceKm']

    # This drops all rows with neg layover time
    data_engineered = drop_neg_layover_time(raw_data)

    # Create the target
    #First ensure ItineraryRedirects is float as zscore doesnt work with int
    # data_engineered['ItineraryRedirects'] = data_engineered['ItineraryRedirects'].astype('float64')
    # Then create the target
    processed_data = target_creation_function(data_engineered, target_func_param1, target_func_param2, target_func_param3)

    # Seperating target so encoders dont store a df shape that is larger than real-world data
    # This is so encoders do not expect the extra column when running on new data, which will not have a target
    y = processed_data[target]

    model_data = processed_data.drop(columns=[target])

    #BINARY ENCODING

    #Binary encoding origin and destination
    o_encoder = ce.BinaryEncoder()
    origin_apt_encoded = o_encoder.fit_transform(model_data['OriginApt'])

    d_encoder = ce.BinaryEncoder()
    destination_apt_encoded = d_encoder.fit_transform(model_data['DestinationApt'])

    # Binary encoding Operator IATA'
    seg_0_encoder = ce.BinaryEncoder()
    seg_0_binary = seg_0_encoder.fit_transform(model_data['Seg_0_OperatingCarrierIATA'])

    seg_1_encoder = ce.BinaryEncoder()
    seg_1_binary = seg_1_encoder.fit_transform(model_data['Seg_1_OperatingCarrierIATA'])

    seg_2_encoder = ce.BinaryEncoder()
    seg_2_binary = seg_2_encoder.fit_transform(model_data['Seg_2_OperatingCarrierIATA'])

    seg_3_encoder = ce.BinaryEncoder()
    seg_3_binary = seg_3_encoder.fit_transform(model_data['Seg_3_OperatingCarrierIATA'])

    #Concatinating newly encoded columns
    origin_binary = pd.concat([model_data, origin_apt_encoded], axis=1)
    dest_binary = pd.concat([origin_binary, destination_apt_encoded], axis=1)
    seg0_bin = pd.concat([dest_binary, seg_0_binary], axis=1)
    seg1_bin = pd.concat([seg0_bin, seg_1_binary], axis=1)
    seg2_bin = pd.concat([seg1_bin, seg_2_binary], axis=1)
    all_binary = pd.concat([seg2_bin, seg_3_binary], axis=1)

    all_binary = all_binary[columns_to_process]

    #SCALING
    # Box cox
    if box_cox_columns == False:
        box_lambda = 0
    else:
        for col in box_cox_columns:
            all_binary[col], box_lambda = stats.boxcox(all_binary[col])

    # Yeo-johnson
    if yeo_johnson_columns == False:
        yeo_lambda = 0
    else:
        for col in yeo_johnson_columns:
            all_binary[col], yeo_lambda = stats.yeojohnson(all_binary[col])

    # Log transformations
    if log_transform_columns == False:
        pass
    else:
        for column in log_transform_columns:
            all_binary.loc[:, column] = np.log1p(model_data[column])

    #Min max scaling
    if min_max_scaling == False:
        minmax_scaler= 0
    else:
        minmax_scaler = MinMaxScaler()
        all_binary[min_max_scaling] = minmax_scaler.fit_transform(all_binary[min_max_scaling])

    # Cyclical encoding
    all_binary['sin_day'] = np.sin(2 * np.pi * all_binary['dayofweek'] / 7)
    all_binary['cos_day'] = np.cos(2 * np.pi * all_binary['dayofweek'] / 7)

    all_binary.drop(columns='dayofweek', inplace=True)

    #Inversing the importance of SelfTransfer, so Non Self Transfer is seen as better by the model
    all_binary['SelfTransfer'] = all_binary['SelfTransfer'].apply(convert_bool_to_num)

    #STORING SCALERS
    class PreprocessScalers:
        def __init__(self, o_encoder, d_encoder, box_lambda, yeo_lambda, minmax_scaler,seg_0_encoder, seg_1_encoder, seg_2_encoder, seg_3_encoder):
                self.o_encoder = o_encoder
                self.d_encoder = d_encoder
                self.box_lambda = box_lambda
                self.yeo_lambda = yeo_lambda
                self.minmax_scaler = minmax_scaler
                self.seg_0_encoder = seg_0_encoder
                self.seg_1_encoder = seg_1_encoder
                self.seg_2_encoder = seg_2_encoder
                self.seg_3_encoder = seg_3_encoder

    scalers = PreprocessScalers(o_encoder, d_encoder, box_lambda, yeo_lambda, minmax_scaler,seg_0_binary, seg_1_binary, seg_2_binary, seg_3_binary)

    #Adding y into dataset
    all_binary[target] = y

    # Returning dataframe and scalers
    return all_binary, scalers

In [86]:
smaller_raw_data = raw_data[:5000].copy()

In [87]:
columns = ['Stops','DurationMin', 'total_layover_time_ratio', 'OriginApt', 'DestinationApt',
            'Total_Flight_Distance','extra_travel_distance_ratio', 'TravelHorizonDays', 'dayofweek',
            'TravelDistanceKm', 'PricePerPax', 'SelfTransfer']

box_cox_columns = ['DurationMin', 'TravelDistanceKm', 'PricePerPax']

yeo_johnson_columns = ['total_layover_time_ratio', 'Total_Flight_Distance', 'extra_travel_distance_ratio']

min_max_scaling = ['TravelHorizonDays','Stops']

In [88]:
smaller_raw_data

,OriginApt,OriginCty,OriginCtry,DestinationApt,DestinationCty,DestinationCtry,TravelHorizonDays,TravelDistanceKm,SelfTransfer,Stops,...,Seg_0_OperatingCarrierIATA,Seg_1_OperatingCarrierIATA,Seg_2_OperatingCarrierIATA,Seg_3_OperatingCarrierIATA,Total_Flight_Distance,Total_Flight_Duration,passengers,PricePerPax,ItineraryRedirects,ODRedirects
0,TLV,TLV,IL,PRG,PRG,CZ,67,2635,False,0,...,LY,,,,2635,240,3,183.202500,2,119
1,TLV,TLV,IL,PRG,PRG,CZ,67,2635,False,0,...,BZ,,,,2635,245,1,162.930000,1,119
2,TLV,TLV,IL,PRG,PRG,CZ,67,2635,False,0,...,LY,,,,2635,250,17,136.458333,12,119
3,TLV,TLV,IL,RHO,RHO,GR,68,794,False,0,...,6H,,,,794,100,23,168.915000,11,42
4,TLV,TLV,IL,RHO,RHO,GR,68,794,False,0,...,6H,,,,794,100,3,172.500000,1,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,TRN,TRN,IT,AMS,AMS,NL,15,818,False,1,...,EN,LH,,,1134,165,2,97.290000,2,19
4996,TRN,TRN,IT,BCN,BCN,ES,68,625,False,0,...,VY,,,,625,85,9,33.975000,3,49
4997,TRN,TRN,IT,BCN,BCN,ES,68,625,False,0,...,FR,,,,625,90,1,37.065000,1,49
4998,TRN,TRN,IT,BCN,BCN,ES,68,625,False,0,...,VY,,,,625,90,4,50.275000,1,49


In [89]:
for column in smaller_raw_data.select_dtypes(include=['int64']).columns:
    smaller_raw_data[column] = smaller_raw_data[column].astype('float64')

In [91]:
df, scal = all_preprocessing(smaller_raw_data, columns, scale_itin_redirects, 'Score_Z_score_0_50',
                  min_max_scaling=min_max_scaling, target_func_param1='ItineraryRedirects', target_func_param2=0, target_func_param3=50)

In [92]:
df

,Stops,DurationMin,total_layover_time_ratio,OriginApt,DestinationApt,Total_Flight_Distance,extra_travel_distance_ratio,TravelHorizonDays,TravelDistanceKm,PricePerPax,SelfTransfer,sin_day,cos_day,Score_Z_score_0_50
0,0.000000,240.0,0.000000,TLV,PRG,2635.0,1.000000,0.744444,2635.0,183.202500,1,-0.974928,-0.222521,7.498985
1,0.000000,245.0,0.000000,TLV,PRG,2635.0,1.000000,0.744444,2635.0,162.930000,1,-0.974928,-0.222521,6.400767
2,0.000000,250.0,0.000000,TLV,PRG,2635.0,1.000000,0.744444,2635.0,136.458333,1,-0.433884,-0.900969,18.481165
3,0.000000,100.0,0.000000,TLV,RHO,794.0,1.000000,0.755556,794.0,168.915000,1,-0.433884,-0.900969,19.484019
4,0.000000,100.0,0.000000,TLV,RHO,794.0,1.000000,0.755556,794.0,172.500000,1,0.433884,-0.900969,6.133944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.333333,555.0,0.702703,TRN,AMS,1134.0,1.386308,0.166667,818.0,97.290000,1,0.781831,0.623490,21.427186
4996,0.000000,85.0,0.000000,TRN,BCN,625.0,1.000000,0.755556,625.0,33.975000,1,-0.781831,0.623490,9.952808
4997,0.000000,90.0,0.000000,TRN,BCN,625.0,1.000000,0.755556,625.0,37.065000,1,-0.433884,-0.900969,5.217688
4998,0.000000,90.0,0.000000,TRN,BCN,625.0,1.000000,0.755556,625.0,50.275000,1,-0.974928,-0.222521,5.217688


In [25]:
smaller_raw_data['OD'] = smaller_raw_data['OriginCty'] + smaller_raw_data['DestinationCty']

# This calculates the total layover time with ratio
smaller_raw_data['total_layover_time'] = smaller_raw_data['DurationMin'] - smaller_raw_data['Total_Flight_Duration']
smaller_raw_data['total_layover_time_ratio'] =smaller_raw_data['total_layover_time'] /smaller_raw_data['DurationMin']

# This calculates the difference between total distance traveled and 'straight line' distance
smaller_raw_data['extra_travel_distance'] = smaller_raw_data['Total_Flight_Distance'] - smaller_raw_data['TravelDistanceKm']
smaller_raw_data['extra_travel_distance_ratio'] =  smaller_raw_data['Total_Flight_Distance'] / smaller_raw_data['TravelDistanceKm']

# This drops all rows with neg layover time
data_engineered = drop_neg_layover_time(smaller_raw_data)


In [30]:
data_engineered

,OriginApt,OriginCty,OriginCtry,DestinationApt,DestinationCty,DestinationCtry,TravelHorizonDays,TravelDistanceKm,SelfTransfer,Stops,...,Total_Flight_Duration,passengers,PricePerPax,ItineraryRedirects,ODRedirects,OD,total_layover_time,total_layover_time_ratio,extra_travel_distance,extra_travel_distance_ratio
0,TLV,TLV,IL,PRG,PRG,CZ,67,2635,False,0,...,240,3,183.202500,2,119,TLVPRG,0,0.0,0,1.0
1,TLV,TLV,IL,PRG,PRG,CZ,67,2635,False,0,...,245,1,162.930000,1,119,TLVPRG,0,0.0,0,1.0
2,TLV,TLV,IL,PRG,PRG,CZ,67,2635,False,0,...,250,17,136.458333,12,119,TLVPRG,0,0.0,0,1.0
3,TLV,TLV,IL,RHO,RHO,GR,68,794,False,0,...,100,23,168.915000,11,42,TLVRHO,0,0.0,0,1.0
4,TLV,TLV,IL,RHO,RHO,GR,68,794,False,0,...,100,3,172.500000,1,42,TLVRHO,0,0.0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,TRN,TRN,IT,AMS,AMS,NL,15,818,False,1,...,165,2,97.290000,2,19,TRNAMS,390,0.702703,316,1.386308
4996,TRN,TRN,IT,BCN,BCN,ES,68,625,False,0,...,85,9,33.975000,3,49,TRNBCN,0,0.0,0,1.0
4997,TRN,TRN,IT,BCN,BCN,ES,68,625,False,0,...,90,1,37.065000,1,49,TRNBCN,0,0.0,0,1.0
4998,TRN,TRN,IT,BCN,BCN,ES,68,625,False,0,...,90,4,50.275000,1,49,TRNBCN,0,0.0,0,1.0
